In [5]:
import cv2
import mediapipe as mp

# MediaPipe Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,               # 동시에 추적할 최대 손의 수
    min_detection_confidence=0.5,  # 손 검출을 위한 최소 신뢰도
    min_tracking_confidence=0.5)   # 손 추적을 위한 최소 신뢰도
mp_drawing = mp.solutions.drawing_utils  # 그리기 함수 초기화

# 웹캠 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    # BGR 이미지를 RGB로 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지 반전
    image = cv2.flip(image, 1)

    # 성능을 위해 이미지 쓰기 불가능으로 설정
    # MediaPipe를 통해 손 검출
    results = hands.process(image)

    # 이미지를 RGB에서 BGR로 변환하여 그리기 작업 준비
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 검출된 손의 랜드마크를 이미지에 그림 및 위치 저장
    hand_landmarks_list = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 랜드마크 그리기
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2))
            # 손의 위치 저장
            hand_landmarks_list.append(hand_landmarks)
            print(hand_landmarks_list)

    # 결과 이미지 표시
    cv2.imshow('Hand Tracking', image)

    # 손의 위치 변수에 저장
    # 여기서 hand_landmarks_list를 사용하여 원하는 위치에 접근하여 변수에 저장할 수 있습니다.
    # 예를 들어 첫 번째 손의 위치를 저장하려면 hand_landmarks_list[0]를 사용할 수 있습니다.

    # 'q'를 눌러 종료
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()




[landmark {
  x: 0.982487142086029
  y: 0.7736784815788269
  z: 2.585198046745063e-07
}
landmark {
  x: 0.9427217841148376
  y: 0.7444488406181335
  z: -0.0939456969499588
}
landmark {
  x: 0.9162974953651428
  y: 0.6591793894767761
  z: -0.11080759763717651
}
landmark {
  x: 0.9044204950332642
  y: 0.5865879058837891
  z: -0.10077796131372452
}
landmark {
  x: 0.9088351726531982
  y: 0.5105350017547607
  z: -0.08294205367565155
}
landmark {
  x: 0.9659916758537292
  y: 0.5302546620368958
  z: -0.07635882496833801
}
landmark {
  x: 0.9617899060249329
  y: 0.4143103361129761
  z: -0.09335757791996002
}
landmark {
  x: 0.9645791053771973
  y: 0.33243948221206665
  z: -0.10729163140058517
}
landmark {
  x: 0.971206784248352
  y: 0.26888084411621094
  z: -0.11399777978658676
}
landmark {
  x: 0.9640694856643677
  y: 0.546279788017273
  z: -0.01960255578160286
}
landmark {
  x: 0.9531030654907227
  y: 0.47278428077697754
  z: -0.02174253575503826
}
landmark {
  x: 0.9558136463165283
  y: 0.

In [6]:
import mediapipe as mp
import cv2

# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False, 
    model_complexity=1, 
    smooth_landmarks=True, 
    enable_segmentation=True, 
    smooth_segmentation=True, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)

# 웹캠 캡처
cap = cv2.VideoCapture(0)

while True:
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:

        # 손의 위치 추출
        # 예시로 손목 (landmark index: 15)의 좌표를 가져와 변수에 저장
        hand_x = results.pose_landmarks.landmark[15].x * image.shape[1]
        hand_y = results.pose_landmarks.landmark[15].y * image.shape[0]

        # 손의 위치 출력
        print("Hand Position (x, y):", hand_x, hand_y)

        # 다른 프로그램으로 손의 위치를 전송
        # 여기서는 예시로 print를 사용하여 콘솔에 출력합니다.
        print("Sending hand position to another program...")
        
        # 연결된 랜드마크 시각화
        mp.solutions.drawing_utils.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)


    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.segmentation_mask is not None:
        # 세그멘테이션 마스크를 이미지 크기에 맞게 조정
        segmentation_mask = results.segmentation_mask
        segmentation_mask = (segmentation_mask > 0.1)  # 임계값 적용
        segmentation_mask = segmentation_mask.astype(float)
        
        # 세그멘테이션 마스크를 3채널로 복제하여 컬러 이미지와 동일한 차원으로 만듦
        mask_three_channel = cv2.merge([segmentation_mask, segmentation_mask, segmentation_mask])
        
        # 마스크를 적용하여 배경을 제거한 이미지 생성
        foreground = cv2.multiply(mask_three_channel, image.astype(float))
        
        cv2.imshow('Foreground', foreground.astype('uint8'))

    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Hand Position (x, y): 531.7464065551758 525.823917388916
Sending hand position to another program...
Hand Position (x, y): 563.182487487793 716.7850685119629
Sending hand position to another program...
Hand Position (x, y): 578.3944702148438 795.571460723877
Sending hand position to another program...
Hand Position (x, y): 573.5185623168945 794.2545318603516
Sending hand position to another program...
Hand Position (x, y): 571.7899703979492 807.2323036193848
Sending hand position to another program...
Hand Position (x, y): 580.7834625244141 820.3692626953125
Sending hand position to another program...
Hand Position (x, y): 579.3302536010742 811.4123153686523
Sending hand position to another program...
Hand Position (x, y): 577.198600769043 802.7169799804688
Sending hand position to another program...
Hand Position (x, y): 578.165397644043 807.5356292724609
Sending hand position to another program...
Hand Position (x, y): 577.2846603393555 812.3074150085449
Sending hand position to anot